数据来自[movielens的ml-1m.zip](https://grouplens.org/datasets/movielens/),wget http://files.grouplens.org/datasets/movielens/ml-1m.zip 后解压到/data/目录下就好

In [1]:
import pandas as pd
import numpy as np
users_columns = ['userid', 'gender', 'age', 'occupationid', 'zip-code']
users = pd.read_csv('./data/users.dat', sep='::', header=None, names=users_columns, engine='python')
users.head(5)

,userid,gender,age,occupationid,zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [2]:
movies_columns = ['movieid', 'title', 'genres']
movies = pd.read_csv('./data/movies.dat', sep='::', header=None, names=movies_columns, engine = 'python')
movies.head(5)

,movieid,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


电影数据的风格类型有以下18种：
1.Action
2.Adventure
3.Animation
4.Children's
5.Comedy
6.Crime
7.Documentary
8.Drama
9.Fantasy
10.Film-Noir
11.Horror
12.Musical
13.Mystery
14.Romance
15.Sci-Fi
16.Thriller
17.War
18.Western

In [3]:
ratings_columns = ['userid','movieid', 'rating', 'timestamp']
ratings = pd.read_csv('./data/ratings.dat', sep='::', header=None, names=ratings_columns, engine = 'python')
ratings.head(5)

,userid,movieid,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
##数据预处理
def panda_one_hot(column,need_df):
    classes = set(pd.unique(need_df[column]))
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, need_df[column].values)),
                             dtype=np.int32)
    onehot_pd = pd.DataFrame(labels_onehot,columns = [i for i in classes_dict.keys()])
    return onehot_pd
def panda_many_hot(set_genres, need_df):
    classes = set(set_genres)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    genres_array = []
    for i in need_df.index:
        genres_array.append(np.array(list(map(classes_dict.get, need_df['genres'].values[i]))).sum(axis = 0))
    labels_onehot = np.array(genres_array)
    onehot_pd = pd.DataFrame(labels_onehot,columns = [i for i in classes_dict.keys()])
    return onehot_pd
def load_data():
    #gender,age,occupationid都是分类变量我们做one-hot处理
    #zip-code没用，删除
    users_columns = ['userid', 'gender', 'age', 'occupationid', 'zip-code']
    users = pd.read_csv('./data/users.dat', sep='::', header=None, names=users_columns, engine='python')
    
    gender = panda_one_hot('gender', users)
    age = panda_one_hot('age', users)
    occupationid = panda_one_hot('occupationid', users)
    users = pd.concat([users, gender, age, occupationid], axis = 1)
    users = users.drop(columns = ['gender', 'age', 'occupationid', 'zip-code'],axis = 1)
    
    #movies的genres做many-hot处理
    movies_columns = ['movieid', 'title', 'genres']
    movies = pd.read_csv('./data/movies.dat', sep='::', header=None, names=movies_columns, engine = 'python')
    movies['genres'] = movies['genres'].apply(lambda x: x.split("|"))
    set_genres = []
    for i in movies.index:
        set_genres += movies['genres'].iloc[i]
    genres = panda_many_hot(set_genres, movies)
    movies = pd.concat([movies, genres], axis = 1)
    #movies = movies.drop(columns = ['genres'], axis = 1) 不删除，可以embdding后使用
    
    ratings_columns = ['userid','movieid', 'rating', 'timestamp']
    ratings = pd.read_csv('./data/ratings.dat', sep='::', header=None, names=ratings_columns, engine = 'python')
    
    return users, movies, ratings

In [7]:
def load_data_embdding():
    # 用于做embedding
    # zip-code没用，删除
    users_columns = ['userid', 'gender', 'age', 'occupationid', 'zip-code']
    users = pd.read_csv('./data/users.dat', sep='::', header=None, names=users_columns, engine='python')

    # gender = panda_one_hot('gender', users)
    # age = panda_one_hot('age', users)
    # occupationid = panda_one_hot('occupationid', users)
    # users = pd.concat([users, gender, age, occupationid], axis = 1)
    users = users.drop(columns=['zip-code'], axis=1)

    movies_columns = ['movieid', 'title', 'genres']
    movies = pd.read_csv('./data/movies.dat', sep='::', header=None, names=movies_columns, engine='python')
    movies['genres'] = movies['genres'].apply(lambda x: x.split("|"))

    ratings_columns = ['userid', 'movieid', 'rating', 'timestamp']
    ratings = pd.read_csv('./data/ratings.dat', sep='::', header=None, names=ratings_columns, engine='python')

    return users, movies, ratings